# 1章 セレクションバイアスとRCT

In [2]:
!pip install rdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install rdd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rdd: filename=rdd-0.0.3-py3-none-any.whl size=4720 sha256=2660da299bc48c5d32c032954c7595b682ab6ed0920399edbd67a08644caa58f
  Stored in directory: /root/.cache/pip/wheels/69/0a/8e/e634d2abb998e7315fd73eb576701a20017f0a2015653304a0
Successfully built rdd


In [6]:
!pip install CausalImpact

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 32.8 MB/s eta 0:00:00


In [10]:
!pip install PropensityScoreMatching

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement PropensityScoreMatching (from versions: none)
ERROR: No matching distribution found for PropensityScoreMatching


In [11]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../scripts/')

import numpy as np
import pandas as pd

import rdata
from rdd import rdd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from causalimpact import CausalImpact
from statsmodels.stats.weightstats import ttest_ind
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

# from propensity_score_matching import PropensityScoreMatching

## 1.4 メールマーケティングの効果検証

### 1.4.1 RCTを行ったデータの準備

In [12]:
email_data = pd.read_csv('http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv')
email_data.head(3)

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0


In [13]:
email_data.shape

(64000, 12)

In [15]:
# 男性向けメール配信とメール配信されなかったサンプルに限定
male_df = email_data[email_data['segment'] != 'Womens E-Mail']
male_df.shape

(42613, 12)

In [17]:
# 介入を表すTreatment変数を追加
male_df['treatment'] = male_df['segment'].map(lambda x: 1 if x == 'Mens E-Mail' else 0)
male_df.head(3)

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0,0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,1
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0.0,1


### 1.4.2 RCTデータの集計と有意差検定

In [20]:
# 集計
summary_by_segment = pd.pivot_table(
    data=male_df,
    values=['conversion', 'spend', 'visit'],
    index=['treatment'],
    aggfunc={'conversion': np.mean, 'spend': np.mean, 'visit': np.ma.count}
)

summary_by_segment.columns = ['conversion_rate', 'spend_mean', 'count']
summary_by_segment

,conversion_rate,spend_mean,count
treatment,,,
0,0.005726,0.652789,21306
1,0.012531,1.422617,21307




*   Count：介入有データと介入無データ数は同じ。
*   Conversion＿rate：介入有データの方がConversion_rate（メール配信してから２週間以内に購入する確率）が高くなっている。介入により購入のきっかけを作り出せていそう。
*   Spend＿mean：介入有データの方が購入した際の購入額が高くなっている。




In [22]:
# 有意差検定
rct_ttest = ttest_ind(
    male_df[male_df['treatment'] == 1]['spend'],
    male_df[male_df['treatment'] == 0]['spend'],
    usevar='pooled' # 母分散は未知だが等しい場合"pooled"を用いる。母分散が等しくない（ウェルチのt検定）を行う場合は"unequal"を入力
)

for key, val in zip(['t', 'p-value', 'df'], rct_ttest):
    print('{} = {}'.format(key, val))

t = 5.300090294465455
p-value = 1.163200872605976e-07
df = 42611.0


p値は非常に小さいことから、介入有と介入無のSpend（購入金額）の差は、統計的に有意であることが確認できた。

### 1.3.4 バイアスのあるデータによる効果の検証

In [25]:
# バイアスのあるデータの準備
# 潜在的に購入意欲が高いと考えられるユーザに対してメールを多く配信。
treatment_data = male_df[male_df['treatment'] == 1]
control_data = male_df[male_df['treatment'] == 0]

# 介入有データでは、「昨年度の購入金額が低い」「Recencyが大きい」「接触チャネルが単体」をランダムに削除。
treatment_biased = treatment_data.drop(treatment_data[~(
    (treatment_data['history'] > 300) |
    (treatment_data['recency'] < 6) |
    (treatment_data['recency'] == 'Multichannel')
)].sample(frac=0.5, random_state=1).index)

# 介入無データでは、「昨年度の購入金額が高い」「Recencyが小さい」「接触チャネルが複数」をランダムに削除。
control_biased = control_data.drop(control_data[
    (control_data['history'] > 300) |
    (control_data['recency'] < 6) |
    (control_data['recency'] == 'Multichannel')
].sample(frac=0.5, random_state=1).index)

biased_data = pd.concat([treatment_biased, control_biased])
biased_data.head(3)

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,1
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0.0,1
13,2,2) $100 - $200,101.64,0,1,Urban,0,Web,Mens E-Mail,1,0,0.0,1


In [26]:
# バイアスのあるデータの集計と有意差の検定
summary_by_segment_biased = pd.pivot_table(
    data=biased_data,
    values=['conversion', 'spend', 'visit'],
    index=['treatment'],
    aggfunc={'conversion': np.mean, 'spend': np.mean, 'visit': np.ma.count}
)
summary_by_segment_biased.columns = ['conversion_rate', 'spend_mean', 'count']

rct_ttest_biased = ttest_ind(
    biased_data[biased_data['treatment'] == 1]['spend'],
    biased_data[biased_data['treatment'] == 0]['spend'],
    usevar='pooled'
)

In [27]:
summary_by_segment_biased

,conversion_rate,spend_mean,count
treatment,,,
0,0.005165,0.634625,14907
1,0.013575,1.560682,17017




*   介入有データと介入無データのconversion_rateとSpend_meanの差が、RCTデータよりも大きくなっている。



In [28]:
for key, val in zip(['t', 'p-value', 'df'], rct_ttest_biased):
    print('{} = {}'.format(key, val))

t = 5.173854873955868
p-value = 2.3069800974507752e-07
df = 31922.0


p値はより小さくなっている。  
->バイアスのあるデータで分析したことにより、平均の差はより広がっているため。

**統計的に有意な差があればどんな結果も正しい効果が推定できていると考えることはできない！**